In [1]:
import sys
sys.path.insert(0, "/Users/artur/workspace/recipedb")
from recipedb.db import get_db

In [2]:
db = get_db()

In [4]:
matched_recipes = list(db.matched_recipes.find())
food_base = list(db.FoodBase.find())

In [42]:
def prepare_features(food_base):
    ingredient_features = list(map(lambda x: x['text'],food_base))
    ingredient_features.sort()
    ingredient_features_indices = {}
    i = 0
    for feature in ingredient_features:
        ingredient_features_indices[feature] = i
        i += 1
    return ingredient_features, ingredient_features_indices, len(ingredient_features)

In [43]:
ingredient_features, ingredient_features_indices, n_features = prepare_features(food_base)

In [22]:
unit_base = list(db.UnitBase.find())

In [44]:
unit_conversion_dict = {}
for unit in unit_base:
    unit_conversion_dict[unit['unit_id']] = unit['conversion']

In [45]:
def convert_unit_to_base(unit_id, qty):
    return unit_conversion_dict[unit_id]*qty

In [48]:
import numpy as np


def process_matched_recipe(recipe):
    Y = np.zeros(n_features)
    matches = recipe['matches']
    for match in matches:
        unit = match['unit']
        matched_text = match['match']['text']
        converted_amount = convert_unit_to_base(unit['unit']['unit_id'], unit['qty'])
        Y[ingredient_features_indices[matched_text]] = converted_amount
    return Y
        

In [50]:
Y = process_matched_recipe(matched_recipes[0])

In [51]:
np.where(Y > 0)

(array([ 214, 1236, 1372, 1852, 1907]),)

In [57]:
ingredient_features[1907]

'toasted pecans'